In [1]:
import numpy as np
import pandas as pd
import os
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from gensim import corpora, models
import gensim
from nltk.util import ngrams 
import emoji
from itertools import chain
paths = "./dataset/"

In [2]:
def get_one_two_three(selects):
    global All_selects
    thgrams_selects = {}
    bigrams_selects = {}
    selects_token ={}
    for select in selects:
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(select)
        tokens = [i for i in tokens if(i.lower() not in stopwords.words('english'))]
        bigrams_t = ngrams(tokens, 2) 
        thgrams_t = ngrams(tokens, 3)
        tokens = list(set(tokens))
        bigrams_t = list(set(bigrams_t))
        thgrams_t = list(set(thgrams_t))
        for t in thgrams_t:
            t = str(t).split("'")
            t = t[1] +" "+ t[3]+" "+t[5]
            if t not in thgrams_selects:
                thgrams_selects[t] = 1
            else:
                thgrams_selects[t] += 1
        for b in bigrams_t:
            b = str(b).split("'")
            b = b[1]+" "+b[3]
            if b not in bigrams_selects:
                bigrams_selects[b] = 1
            else:
                bigrams_selects[b] += 1
        for token in tokens:
            if token not in selects_token:
                selects_token[token] = 1
            else:
                selects_token[token] += 1
    df_selects_token = pd.DataFrame.from_dict(selects_token,orient="index",columns=["频率"]).sort_values(by="频率",ascending=False)
    df_selects_token = df_selects_token[df_selects_token["频率"]>=3]
    df_bigrams_selects = pd.DataFrame.from_dict(bigrams_selects,orient="index",columns=["频率"]).sort_values(by="频率",ascending=False)
    df_bigrams_selects = df_bigrams_selects[df_bigrams_selects["频率"]>=2]
    df_thgrams_selects = pd.DataFrame.from_dict(thgrams_selects,orient="index",columns=["频率"]).sort_values(by="频率",ascending=False)
    df_thgrams_selects = df_thgrams_selects[df_thgrams_selects["频率"]>=2]
    return df_selects_token,df_bigrams_selects,df_thgrams_selects

In [3]:
def get_df_out(titles,comments):
    df_titles_token,df_bigrams_titles,df_thgrams_titles = get_one_two_three(titles)
    df_comments_token,df_bigrams_comments,df_thgrams_comments = get_one_two_three(comments)
    df_titles_token['标题-1字']=df_titles_token.index
    df_titles_token = df_titles_token.reset_index(drop=True)
    df_bigrams_titles['标题-2字']=df_bigrams_titles.index
    df_bigrams_titles = df_bigrams_titles.reset_index(drop=True)
    df_thgrams_titles['标题-3字']=df_thgrams_titles.index
    df_thgrams_titles = df_thgrams_titles.reset_index(drop=True)
    df_comments_token['评论-1字']=df_comments_token.index
    df_comments_token = df_comments_token.reset_index(drop=True)
    df_bigrams_comments['评论-2字']=df_bigrams_comments.index
    df_bigrams_comments = df_bigrams_comments.reset_index(drop=True)
    df_thgrams_comments['评论-3字']=df_thgrams_comments.index
    df_thgrams_comments = df_thgrams_comments.reset_index(drop=True)
    if df_titles_token.empty:
        df_t = pd.DataFrame()
    elif df_bigrams_titles.empty:
        df_t = df_titles_token
    elif df_thgrams_titles.empty:
        df_t = pd.concat([df_titles_token,df_bigrams_titles],axis=1)
    else:
        df_t = pd.concat([df_titles_token,df_bigrams_titles,df_thgrams_titles],axis=1)
    if df_thgrams_comments.empty:
        df_c = pd.concat([df_comments_token,df_bigrams_comments],axis=1)
    else:
        df_c = pd.concat([df_comments_token,df_bigrams_comments,df_thgrams_comments],axis=1)
    df_out = pd.concat([df_t,df_c],axis=1)
    return df_out

In [4]:
def change_coloum(df):
    df.columns = [x+"-"+str(i) if x=="频率" else x for i,x in enumerate(df.columns)]
    a,b,c = [],[],[]
    for i,j in enumerate(list(df.columns)):
        if i%2==0:
            a.append(j)
        else:
            b.append(j)
    for i in range(int(len(list(df.columns))/2)):
        c.append(b[i])
        c.append(a[i])
    df = df[c]
    return df

In [5]:
df_tmp = pd.DataFrame()
for _,_,files in os.walk(paths):
    print(files)
writer = pd.ExcelWriter("final.xlsx")
all_titles = []
all_comments = []
for file in files:
    keyname = file.split("-")[0]
    path = paths+file
    df = pd.read_excel(path)
    df = df[["标题","内容"]]
    titles = []
    comments = []
    df.标题.apply(lambda x:titles.append(x))
    df.内容.apply(lambda x:comments.append(x))
    titles = [emoji.demojize(str(i)).lower() for i in titles]
    comments = [emoji.demojize(str(i)).lower() for i in comments]
    all_titles.append(titles)
    all_comments.append(comments)
    df_out = get_df_out(titles,comments)
    df_out = change_coloum(df_out)
    df_out.to_excel(writer,sheet_name=keyname+"-"+str(len(df)),index=None)
df_out = get_df_out(list(chain.from_iterable(all_titles)),list(chain.from_iterable(all_comments)))
df_out = change_coloum(df_out)
df_out.to_excel("all_titles_comments.xlsx",sheet_name="所有标题和评论词频汇总",index=None)
#     print(df_out.columns)
writer.close()
test = pd.read_excel("final.xlsx",None)
writer = pd.ExcelWriter("final-out.xlsx")
for key in test.keys():
    pd_out = test[key]
    pd_out = pd_out.head(100)
    pd_out.to_excel(writer,sheet_name=key,index=None)
writer.close()

['B074737RWR-US-Reviews-200930.xlsx', 'B07474J9BX-US-Reviews-200930.xlsx', 'B07DC7FVND-US-Reviews-200930.xlsx', 'B07GM4TBQC-US-Reviews-200930.xlsx', 'B07MDLFKRW-US-Reviews-200930.xlsx', 'B07PGF62GC-US-Reviews-200930.xlsx', 'B07QH7LFK1-US-Reviews-200930.xlsx', 'B07SZC274F-US-Reviews-200930.xlsx', 'B07TC3L44B-US-Reviews-200930.xlsx', 'B07WZLPCLQ-US-Reviews-200930.xlsx', 'B082WR58HY-US-Reviews-200930.xlsx']
